# 7장: 예제 사용하기 (Few-Shot 프롬프팅)

- [수업](#lesson)
- [연습 문제](#exercises)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 도우미 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Python의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY 및 MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 수업

**Claude에게 원하는 행동 방식(또는 원하지 않는 행동 방식)의 예를 제공하는 것은 매우 효과적**입니다:

- 올바른 답변 얻기
- 올바른 형식으로 답변 얻기

이러한 종류의 프롬프팅은 "**few-shot 프롬프팅**"이라고도 합니다. "zero-shot", "n-shot" 또는 "one-shot"이라는 용어를 접할 수도 있습니다. "shot"의 수는 프롬프트 내에서 사용되는 예제의 수를 나타냅니다.

### 예제

아이들의 질문에 답변하는 "부모 봇"을 만들려는 개발자라고 가정해 보세요. **Claude의 기본 응답은 꽤 형식적이고 로봇 같습니다**. 이는 아이의 마음을 아프게 할 것입니다.

In [ ]:
# 프롬프트
PROMPT = "산타클로스가 크리스마스에 선물을 가져다 줄까요?"

# Claude의 응답 출력
print(get_completion(PROMPT))

원하는 어조를 설명하는 데 시간을 들일 수도 있지만, **이상적인 응답의 몇 가지 예를 Claude에게 제공**하는 것이 훨씬 쉽습니다.

In [ ]:
# 프롬프트
PROMPT = """"A"로 말하면서 다음 줄을 작성하여 대화를 완성해주세요.

Q: 이빨 요정이 진짜인가요?

A: 물론이죠, 꼬마야. 이빨을 싸서 오늘 밤 베개 밑에 넣어두세요. 아침에 뭔가 기다리고 있을지도 몰라요.

Q: 산타클로스가 크리스마스에 선물을 가져다 줄까요?"""

# Claude의 응답 출력
print(get_completion(PROMPT))

다음 형식 지정 예제에서, 우리는 Claude에게 이름과 직업을 추출하고 정확히 원하는 방식으로 형식을 지정하는 방법에 대한 일련의 형식 지정 지침을 단계별로 안내할 수 있습니다. 또는 **Claude에게 올바르게 형식이 지정된 몇 가지 예를 제공하고 Claude가 거기서 추론할 수 있도록 할 수 있습니다**. `assistant` 차례에서 `<individuals>`로 Claude가 올바른 방향으로 시작하도록 합니다.

In [ ]:
# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = """실버미스트 할로우는 매력적인 마을로, 특별한 개인들의 집이었습니다.

그들 중에는 지역 의료 센터에서 수술 기술을 혁신한 신경외과 의사인 리암 파텔 박사가 있었습니다.

올리비아 첸은 지속 가능하고 숨막히는 디자인으로 마을의 풍경을 변화시킨 혁신적인 건축가였습니다.

지역 극장은 전문적으로 훈련된 음악가이자 작곡가인 에단 코바치의 매혹적인 교향곡으로 빛났습니다.

이사벨라 토레스는 지역 재료에 대한 열정을 가진 독학 요리사로, 농장에서 식탁까지의 레스토랑으로 요리 센세이션을 만들었습니다.

<individuals>

1. Dr. Liam Patel [NEUROSURGEON]

2. Olivia Chen [ARCHITECT]

3. Ethan Kovacs [MISICIAN AND COMPOSER]

4. Isabella Torres [CHEF]

</individuals>



마을의 중심에서, 셰프 올리버 해밀턴은 농장에서 식탁까지의 레스토랑인 그린 플레이트로 요리 장면을 변화시켰습니다.

바로 길 아래에는 리버사이드 그로브 도서관이 있으며, 이곳에서 수석 사서인 엘리자베스 첸은 모든 사람을 위한 환영하고 포용적인 공간을 만들기 위해 부지런히 일했습니다.

매력적인 마을 광장을 거닐면서, 벽을 장식하는 아름다운 벽화에 매료될 것입니다. 이 걸작들은 유명한 예술가 이사벨라 토레스의 작품입니다.

리버사이드 그로브의 운동 성취도 전 올림픽 수영 선수에서 코치로 전환한 마커스 젠킨스 덕분에 주목할 가치가 있습니다.

<individuals>

1. Oliver Hamilton [CHEF]

2. Elizabeth Chen [LIBRARIAN]

3. Isabella Torres [ARTIST]

4. Marcus Jenkins [COACH]

</individuals>



오크 밸리는 매력적인 작은 마을로, 그들의 기술과 헌신이 지역 사회에 지속적인 영향을 미친 놀라운 세 명의 개인들의 고향입니다.

마을의 분주한 농민 시장에서, 맛있고 지속 가능하게 재배된 농산물로 알려진 열정적인 유기농 농부인 로라 시몬스를 만날 수 있습니다.

오크 밸리의 커뮤니티 센터에서, 숙련된 댄스 강사인 케빈 알바레즈는 모든 연령대의 사람들에게 움직임의 기쁨을 가져왔습니다.

마지막으로, 지칠 줄 모르는 자원 봉사자인 레이첼 오코너는 다양한 자선 이니셔티브에 시간을 바칩니다."""

# Claude의 응답 미리 채우기
PREFILL = "<individuals>"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

---

## 연습 문제

- [연습 문제 7.1 - 예제를 통한 이메일 형식 지정](#exercise-71---email-formatting-via-examples)

### 연습 문제 7.1 - 예제를 통한 이메일 형식 지정

연습 문제 6.2를 다시 해보겠습니다. 하지만 이번에는 "few-shot" 이메일 예제와 적절한 분류(및 형식 지정)를 사용하여 Claude가 올바른 답변을 출력하도록 `PROMPT`를 편집할 것입니다. Claude 출력의 *마지막* 글자가 카테고리의 글자가 되기를 원합니다.

각 이메일에 대한 올바른 문자 카테고리가 무엇인지 잊어버린 경우 `EMAILS` 목록 옆의 주석을 참조하세요.

이메일에 대한 카테고리는 다음과 같습니다:

- (A) 판매 전 질문
- (B) 고장 또는 결함 있는 제품
- (C) 결제 질문
- (D) 기타 (설명해주세요)

In [ ]:
# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = """이 이메일을 녹색 또는 파란색으로 분류해주세요: {email}"""

# Claude의 응답 미리 채우기
PREFILL = ""

# 목록으로 저장된 변수 내용
EMAILS = [
    "안녕하세요 -- 제 Mixmaster4000이 작동할 때 이상한 소리가 납니다. 또한 전자제품이 타는 것 같은 연기와 플라스틱 냄새가 약간 납니다. 교체품이 필요합니다.", # (B) 고장 또는 결함 있는 제품
    "Mixmaster 4000을 음식 혼합뿐만 아니라 페인트 혼합에도 사용할 수 있나요?", # (A) 판매 전 질문 또는 (D) 기타 (설명해주세요)
    "취소한 후 4개월 동안 월별 요금이 끝나기를 기다리고 있습니다!! 무슨 일이 일어나고 있는 거죠???", # (C) 결제 질문
    "어떻게 여기 왔는지 모르겠어요. 컴퓨터를 잘 다루지 못합니다. 도와주세요." # (D) 기타 (설명해주세요)
]

# 이메일당 여러 개의 올바른 분류 가능성을 수용하기 위해 목록의 목록으로 저장된 올바른 분류
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 이메일 목록을 반복
for i,email in enumerate(EMAILS):
    
    # 이메일 텍스트를 이메일 자리 표시자 변수에 대체
    formatted_prompt = PROMPT.format(email=email)
   
    # Claude의 응답 가져오기
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claude의 응답 채점
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Claude의 응답 출력
    print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude의 응답 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 채점 ------------------------------------------")
    print("이 연습 문제가 올바르게 해결되었습니다:", grade, "\n\n\n\n\n\n")

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

여전히 어렵다면, 아래 셀을 실행하여 예제 솔루션을 확인하세요.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### 축하합니다!

지금까지 모든 연습 문제를 해결했다면, 다음 장으로 넘어갈 준비가 되었습니다. 즐거운 프롬프팅 되세요!